In [26]:
class MulLayer:
  """곱셈 계층
  """
  def __init__(self):
    self.x = None
    self.y = None
  
  def forward(self, x, y):
    self.x = x
    self.y = y
    return x * y
  
  def backward(self, dout):
    dx = dout * self.y
    dy = dout * self.x
    return dx, dy

In [27]:
"""
곱셉 계층의 순전파, 역전파 테스트
"""

apple = 100
apple_num = 2
tax_rate = 1.1

mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax_rate)
print(price)

# 역전파
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)
print(dprice, dapple_price, dtax, dapple, dapple_num)

220.00000000000003
1 1.1 200 2.2 110.00000000000001


In [28]:
class AddLayer:
  def __init__(self):
    pass

  def forward(self, x, y):
    return x + y
  
  def backward(self, dout):
    dx = dout * 1
    dy = dout * 1
    return dx, dy

In [29]:
"""
곱셈, 덧셈 계층의 순전파, 역전파 테스트
"""

apple_price = 100
apple_num = 2
orange_price = 150
orange_num = 3
tax_rate = 1.1

# 계층들
mul_apple_price = MulLayer()
mul_orange_price = MulLayer()
add_apple_orange_price = AddLayer()
mul_tax = MulLayer()

# 순전파
apple_total_price = mul_apple_price.forward(apple_price, apple_num)
orange_total_price = mul_orange_price.forward(orange_price, orange_num)
total_price = add_apple_orange_price.forward(apple_total_price, orange_total_price)
final_price = mul_tax.forward(total_price, tax_rate)
print("apple_price:", apple_price)
print("apple_num:", apple_num)
print("orange_price:", orange_price)
print("orange_num:", orange_num)
print("tax_rate:", tax_rate)
print("apple_total_price:", apple_total_price)
print("orange_total_price:", orange_total_price)
print("total_price:", total_price)
print("final_price:", final_price)

# 역전파
d_final_price = 1
d_total_price, d_tax = mul_tax.backward(d_final_price)
d_apple_total_price, d_orange_total_price = add_apple_orange_price.backward(d_total_price)
d_apple_price, d_apple_num = mul_apple_price.backward(d_apple_total_price)
d_orange_price, d_orange_num = mul_orange_price.backward(d_orange_total_price)
print("d_final_price:", d_final_price)
print("d_total_price:", d_total_price)
print("d_tax:", d_tax)
print("d_apple_total_price:", d_apple_total_price)
print("d_orange_total_price:", d_orange_total_price)
print("d_apple_price:", d_apple_price)
print("d_apple_num:", d_apple_num)
print("d_orange_price:", d_orange_price)
print("d_orange_num:", d_orange_num)


apple_price: 100
apple_num: 2
orange_price: 150
orange_num: 3
tax_rate: 1.1
apple_total_price: 200
orange_total_price: 450
total_price: 650
final_price: 715.0000000000001
d_final_price: 1
d_total_price: 1.1
d_tax: 650
d_apple_total_price: 1.1
d_orange_total_price: 1.1
d_apple_price: 2.2
d_apple_num: 110.00000000000001
d_orange_price: 3.3000000000000003
d_orange_num: 165.0
